In [58]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [67]:
# dummy data
# row 1 -> group 1 etc.
# col 1 -> proj 1 etc.
# 1 is top rank
d1 = np.matrix([[3,2,1],[3,2,1],[1,2,3]])

raw_data = pd.read_csv("Example MAT210 Project Rank Form (Responses) - Form Responses 1.csv")

group_names = raw_data["Names of Group Members"]

loop = len(raw_data.columns)

d = pd.DataFrame({})

for i in range(3, loop):
    col_name = raw_data.columns[i]
    
    d[col_name] = raw_data[col_name]
    
matrix = np.array(d)

for i in range(len(matrix)):
    print(group_names[i] + str("'s rankings were: ") + str(matrix[i]))

Daniel, Nathan, Rachel's rankings were: [ 2  1  4  3  5  6  9  8  7 10 13 12 11 14 15]
daniel's rankings were: [ 1  3  2  5  4  7  6  9  8 11 10 13 12 15 14]
nate1's rankings were: [ 9  6  8  7 10 12 11 13 15  1  5  3  4 14  2]
dan's rankings were: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
nate2's rankings were: [15 11 14 10 13  9 12  8  1  3  5  7  6  4  2]
dan2's rankings were: [15  1 14  2 13  3 12  4 11  5 10  6  9  7  8]
nate3's rankings were: [ 1  3  5  7  9 11 13 15 14 12 10  8  6  4  2]
rachel2's rankings were: [ 3  2  4  5  7  6  8  9 11 10 12 14 13 15  1]
rachel3's rankings were: [ 1  3  4  2  5  6  9 10  7 11  8 12 13 15 14]
nate4's rankings were: [ 1  6 11  2  7 12  3  8 13  4  9 14  5 10 15]
dan3's rankings were: [ 6  5  4  3  2  1  7  8  9 10 11 15 14 13 12]
rachel4's rankings were: [15 14 13 10 11 12  1  9  8  7  4  5  6  3  2]


In [36]:
# Create a new model
m = gp.Model("mip1")

n = len(d)

In [37]:
# Create variables

# assignment matrix
var = m.addVars(n, n, vtype=GRB.BINARY, name='G')

# rating matrix
dg = m.addVars(n, n, vtype=GRB.INTEGER, name='D')

In [38]:
for a in range(n):
    for b in range(n):
        dg[a,b] = d[a,b]*var[a,b]

# Add constraints
m.addConstrs((var.sum(i, '*') == 1
                 for i in range(n)
                 for j in range(n)), name='C')

m.addConstrs((var.sum('*', j) <= 1
                 for i in range(n)
                 for j in range(n)), name='R')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>}

In [39]:
# Set objective
m.setObjective(dg.sum(), GRB.MINIMIZE)

In [40]:
    # Limit how many solutions to collect
    m.setParam(GRB.Param.PoolSolutions, 3)
#     # Limit the search space by setting a gap for the worst possible solution
#     # that will be accepted
#     model.setParam(GRB.Param.PoolGap, 0.10)
#     # do a systematic search for the k-best solutions
#     model.setParam(GRB.Param.PoolSearchMode, 2)

Changed value of parameter PoolSolutions to 3
   Prev: 10  Min: 1  Max: 2000000000  Default: 10


In [41]:
# Optimize model
m.optimize()

Optimize a model with 18 rows, 18 columns and 54 nonzeros
Variable types: 0 continuous, 18 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 12 rows and 9 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)

Root relaxation: objective 4.000000e+00, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.0000000    4.00000  0.00%     -    0s

Explored 0 nodes (4 simplex iterations) in 0.02 seconds
Thread count was 6 (of 6 available processors)

Solution count 2: 4 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+

In [47]:
m.write("model.lp")

nSolutions = m.SolCount
print('Number of solutions found: ' + str(nSolutions))

solution = m.getAttr("X", var)

for i in range(n):
    sol = ''
    for j in range(n):
        if solution[i, j] > 0.5:
            sol += "Group " + str(i+1) +" gets project " + str(j+1)
    print(sol)

Number of solutions found: 2
Group 1 gets project 2
Group 2 gets project 3
Group 3 gets project 1


In [55]:
for e in range(nSolutions):
    m.setParam(GRB.Param.SolutionNumber, e)
    print('%g ' % m.PoolObjVal, end='')

Changed value of parameter SolutionNumber to 0
   Prev: 1  Min: 0  Max: 2000000000  Default: 0
4 Changed value of parameter SolutionNumber to 1
   Prev: 0  Min: 0  Max: 2000000000  Default: 0
6 

6.0